In [219]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random
from copy import deepcopy

In [220]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [221]:
selected_model = Supported_modles.SGD_classifier

In [222]:
clients, test_x, test_y = utils.set_data(selected_model, True, False, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
439683
251723


In [224]:
f1_before = []
for client in clients:
    client.init_empty_model(0.00001)
    try:
        client.train_model(epochs=100)
    except ValueError:
        score = 0
        print(client.name)
        f1_before.append(score)
        continue 
    try: 
        score = client.test_model_f1(y_test=test_y, X_test=test_x)
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

0.7691973498425111
0.5005076142131979
0.5864429659627809
0.761456001884792
0.960560911481157


In [225]:
fedavg = Fedavg("global", selected_model)

Waitiing for a Connection..


In [209]:
clients.remove(clients[0])


In [226]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True
epochs = 100

for client in clients:
    client.init_empty_model(0.00001,epochs)
    X_train, X_test, y_train, y_test = client.split_data(0.8)

    # X_train = client.x_chunks[0]
    # y_train = client.y_chunks[0]

    client.train_model(X_train,y_train,epochs=epochs)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.init_global_model(client.model)
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.
.
.


In [227]:
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.8053521837919718


In [228]:
number_of_rounds = 4
epochs = 100
max_score = 0
optimal_model = None


for round in range(number_of_rounds):
    print(round +1, end=' ')

    applicable_clients = random.sample((clients), len(clients))#random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        print(f'.', end='')

        X_train, X_test, y_train, y_test = client.split_data(0.8)

        # X_train = client.x_chunks[round + 1]
        # y_train = client.y_chunks[round + 1]

       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        client.load_global_model(fedavg.model)
        client.train_local_agent(X_train, y_train, epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
    print(score)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

1 .....0.8513164193393968
0.8513164193393968
2 .....0.7622218328368671
3 .....0.9879518072289157
0.9879518072289157
4 .....0.7530864197530864
5 .....0.9865020403892435


In [196]:
# x = test_x
# y_hat = optimal_model.predict(x)
# score = f1_score(test_y,y_hat)
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)
fedavg.model = optimal_model
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.7165837964317052
0.7291948510648638


In [40]:
for i in f1_before:
    print(f'Difference {score-i}')

Difference 0.15089891751031836
Difference 0.1428049966380307
Difference 0.07722927909559152


## Centralized Zone

In [ ]:
client1 = Client("node1","0.0.0.0", 5001, selected_model)
client1.x = clients[0].x
client1.init_empty_model(0.01)


for client in clients:
    client1.train_model(x=client.x,y=client.y,epochs=200)
    print(f'.')

Creating node1.
.
.
.
.
.


In [ ]:
score = client1.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.8262882168694737
